In [ ]:
from google.colab import files

uploaded = files.upload()

Saving tweets_AlexandarMKD.csv to tweets_AlexandarMKD.csv
Saving tweets_apasiev.csv to tweets_apasiev.csv
Saving tweets_gogamkd.csv to tweets_gogamkd.csv
Saving tweets_GruevskiNikola.csv to tweets_GruevskiNikola.csv
Saving tweets_levica_partija.csv to tweets_levica_partija.csv
Saving tweets_mickoski.csv to tweets_mickoski.csv
Saving tweets_o_spasovski.csv to tweets_o_spasovski.csv
Saving tweets_SDSMakedonija.csv to tweets_SDSMakedonija.csv
Saving tweets_Sekerinska.csv to tweets_Sekerinska.csv
Saving tweets_SPendarovski.csv to tweets_SPendarovski.csv
Saving tweets_TimcoMucunski.csv to tweets_TimcoMucunski.csv
Saving tweets_VFilipche.csv to tweets_VFilipche.csv
Saving tweets_VlatkoGjorchev.csv to tweets_VlatkoGjorchev.csv
Saving tweets_VMRO_DPMNE.csv to tweets_VMRO_DPMNE.csv


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', None)

df1 = pd.read_csv('tweets_AlexandarMKD.csv')
df2 = pd.read_csv('tweets_apasiev.csv')
df3 = pd.read_csv('tweets_gogamkd.csv')
df4 = pd.read_csv('tweets_GruevskiNikola.csv')
df5 = pd.read_csv('tweets_levica_partija.csv')
df6 = pd.read_csv('tweets_mickoski.csv')
df7 = pd.read_csv('tweets_o_spasovski.csv')
df8 = pd.read_csv('tweets_SDSMakedonija.csv')
df9 = pd.read_csv('tweets_Sekerinska.csv')
df10 = pd.read_csv('tweets_SPendarovski.csv')
df11 = pd.read_csv('tweets_TimcoMucunski.csv')
df12 = pd.read_csv('tweets_VFilipche.csv')
df13 = pd.read_csv('tweets_VlatkoGjorchev.csv')
df14 = pd.read_csv('tweets_VMRO_DPMNE.csv')


df = pd.concat([df1,df2,df3,df4,df5,df6,df7,df8,df9,df10,df11,df12,df13,df14],ignore_index=True)
df['tweet.user.screen_name'].value_counts()

,count
tweet.user.screen_name,
apasiev,760
levica_partija,753
SPendarovski,617
VFilipche,600
o_spasovski,600
Sekerinska,600
gogamkd,505
VMRO_DPMNE,500
MickoskiHM,393


In [ ]:
df['retweeters'] = df['retweeters'].fillna('None')
df = df.drop_duplicates(subset=['tweet.full_text'])
df = df.dropna(subset=['tweet.full_text'])

In [ ]:
!pip install -q transformers

In [ ]:
import re
import nltk
from nltk.tokenize import word_tokenize
from collections import Counter
from math import log2
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using device: {device}')

Using device: cuda


In [ ]:
from transformers import MarianMTModel, MarianTokenizer

model_name = 'Helsinki-NLP/opus-mt-mk-en'
translator_tokenizer = MarianTokenizer.from_pretrained(model_name)
translator_model = MarianMTModel.from_pretrained(model_name).to(device)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/1.00M [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/799k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.35M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


pytorch_model.bin:   0%|          | 0.00/306M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

In [ ]:
def batch_translate(texts, batch_size=100):
    translations = []
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]
        tokens = translator_tokenizer(batch, return_tensors='pt', padding=True, truncation=True).to(device)
        translated = translator_model.generate(**tokens)
        batch_translations = [translator_tokenizer.decode(t, skip_special_tokens=True) for t in translated]
        translations.extend(batch_translations)
    return translations

df['translated_text'] = batch_translate(df['tweet.full_text'].astype(str).tolist())

In [ ]:
df.to_csv('translated_tweets.csv')

In [ ]:
df['engagement'] = df['tweet.retweet_count'] + df['tweet.reply_count'] + df['tweet.favorite_count']

In [ ]:
import re
df['clean_text'] = df['translated_text'].apply(lambda x: re.sub(r'http\S+|www\.t\S+|www\.\S+', 'URL', x))

In [ ]:
emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags
                           u"\U00002700-\U000027BF"  # dingbats
                           "]+", flags=re.UNICODE)

df['clean_text'] = df['clean_text'].apply(lambda x: emoji_pattern.sub('', x).replace('\n', ' '))

In [ ]:
nltk.download('punkt_tab')

def lexical_diversity(text):
    tokens = word_tokenize(text)
    if len(tokens) == 0:
        return 0
    return len(set(tokens)) / len(tokens)

df['lexical_diversity'] = df['clean_text'].apply(lexical_diversity)

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [ ]:
def entropy(text):
    tokens = word_tokenize(text)
    if len(tokens) == 0:
        return 0
    counts = Counter(tokens)
    total = len(tokens)
    return -sum((count/total) * log2(count/total) for count in counts.values())

df['entropy'] = df['clean_text'].apply(entropy)

In [ ]:
df.to_csv('transled_tweets2.csv')

In [ ]:
df = df.drop(columns=['tweet_count','tweet.full_text','tweet.retweet_count','tweet.reply_count','tweet.favorite_count','tweet.lang'])

In [ ]:
df = df.drop(columns='translated_text')

In [ ]:
df['clean_text'] = df['clean_text'].apply(lambda x: re.sub(r'(\. ?)+', '. ', x).strip())

In [ ]:
from transformers import pipeline

classifier = pipeline(
    "zero-shot-classification",
    model="joeddav/xlm-roberta-large-xnli",
    device=0
)


config.json:   0%|          | 0.00/734 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

Some weights of the model checkpoint at joeddav/xlm-roberta-large-xnli were not used when initializing XLMRobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

Device set to use cuda:0


In [ ]:
df.to_csv('transled_tweets3.csv')

In [ ]:
batch_size = 100
labels = ["valid", "misleading", "invalid"]

results = []

for i in range(0, len(df), batch_size):
    batch_texts = df['clean_text'].iloc[i:i+batch_size].tolist()
    output = classifier(batch_texts, candidate_labels=labels)
    if isinstance(output, dict):
        batch_labels = [output['labels'][0]]
    else:
        batch_labels = [res['labels'][0] for res in output]
    results.extend(batch_labels)

df['misinformation_label'] = results

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


In [ ]:
df['misinformation_label'].value_counts()

,count
misinformation_label,
valid,6088
invalid,594
misleading,366


In [ ]:
df['affiliated'] = 'unknown'

In [ ]:
df.loc[df['tweet.user.screen_name'] == 'SPendarovski', 'affiliated'] = 'center-left'
df.loc[df['tweet.user.screen_name'] == 'VFilipche', 'affiliated'] = 'center-left'

df.loc[df['tweet.user.screen_name'] == 'VMRO_DPMNE', 'affiliated'] = 'right'
df.loc[df['tweet.user.screen_name'] == 'MickoskiHM', 'affiliated'] = 'right'
df.loc[df['tweet.user.screen_name'] == 'AMiloshoski', 'affiliated'] = 'right'
df.loc[df['tweet.user.screen_name'] == 'GruevskiNikola', 'affiliated'] = 'right'
df.loc[df['tweet.user.screen_name'] == 'gogamkd', 'affiliated'] = 'right'
df.loc[df['tweet.user.screen_name'] == 'TimcoMucunski', 'affiliated'] = 'right'
df.loc[df['tweet.user.screen_name'] == 'AlexandarMKD', 'affiliated'] = 'right'
df.loc[df['tweet.user.screen_name'] == 'VlatkoGjorchev', 'affiliated'] = 'right'

df.loc[df['tweet.user.screen_name'] == 'SDSMakedonija', 'affiliated'] = 'center-left'
df.loc[df['tweet.user.screen_name'] == 'Sekerinska', 'affiliated'] = 'center-left'
df.loc[df['tweet.user.screen_name'] == 'o_spasovski', 'affiliated'] = 'center-left'

df.loc[df['tweet.user.screen_name'] == 'apasiev', 'affiliated'] = 'left'
df.loc[df['tweet.user.screen_name'] == 'levica_partija', 'affiliated'] = 'left'

df['affiliated'].value_counts()

,count
affiliated,
right,2921
center-left,2614
left,1513


In [ ]:
df.to_csv('translated_labeled_tweets.csv')